In [8]:
# Assignment: Open Data Policies comments scrape

import re
import util
import bs4
import queue
import json
import sys
import csv
import urllib
import collections
import pandas as pd
import nltk
from nltk.book import *
import sys
#nltk.download()
from nltk.corpus import stopwords
#from textblob import TextBlob

In [3]:
from textblob import TextBlob

ModuleNotFoundError: No module named 'textblob'

In [ ]:
stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '.', 'the', ',', 'and', 'of', ';', 'for', 'that', 'on', 'to', 'a', 'or', 'is', 'in', '’', '"'])

In [ ]:
starting_urls_list = ["https://mymadison.io/documents/city-of-buffalo-open-data-policy", 
                      "https://mymadison.io/documents/durham-open-data-policy",
                      "https://mymadison.io/documents/city-of-tyler-data-policy",
                      "https://mymadison.io/documents/city-of-glendale-draft-open-data-resolution",
                      "https://mymadison.io/documents/metro-nashville-government-open-data-policy",
                      "https://mymadison.io/documents/city-of-syracuse-open-data-policy",
                      "https://mymadison.io/documents/city-of-buffalo-open-data-policy",
                      "https://mymadison.io/documents/napervilleopendatapolicy",
                      "https://mymadison.io/documents/bart-open-data-policy",
                      "https://mymadison.io/documents/san-francisco-open-data-legislation-2014"]

In [ ]:
starting_url = "https://mymadison.io/documents/city-of-buffalo-open-data-policy?comment_page=1"

## Auxillary functions

In [ ]:
def simple_crawl(starting_url):
    '''
    Scrapes the full webpage from a starting url.

    Inputs:
        starting_url: string
    Returns:
        a soup object
    '''    
    page = urllib.request.urlopen(starting_url)
    if page is not None:
        soup = bs4.BeautifulSoup(page, "lxml")
        return soup

In [ ]:
def comment_id(comment):
    comment_id = comment.get('id')
    return comment_id

In [ ]:
def name(comment):
    name_date = comment.find("div", class_="media-body media-middle")
    name = name_date.find("span")
    name_final = name.string.strip()
    return name_final, name_date

In [ ]:
def datetime(comment):
    name_date = name(comment)[1]
    time = name_date.find("time")
    datetime_final = time.get("datetime")[:-6]
    datetime_final_1 = datetime_final.replace("T", " ")
    return datetime_final_1

In [ ]:
def likes_count(comment):
    name_date = name(comment)[1]
    likes = name_date.find("span", class_="action-count")
    return likes.string.strip()

In [ ]:
def quoted_comment(comment):
    comments = comment.find("div", class_="comment-content")
    quoted_comment = comments.find("blockquote")
    if quoted_comment is not None:
        quoted_comment_final = quoted_comment.text.strip()
        return quoted_comment_final, comments
    else:
        return None, comments

In [ ]:
def actual_comment(comment):
    comments = comment.find("div", class_="comment-content")
    actual_comments = comments.find_all("p", recursive=False)
    
    complete_str_aux = ''
    complete_str_str = ''
    
    for child in actual_comments:
        complete_str_aux += str(child)
        complete_str_str += str(child.text)
    return complete_str_aux, complete_str_str

In [ ]:
def reply_ids(comment):
    
    replies = comment.find_all("div", class_="comment")
    if replies is not None:
        reply_id_list = []
        for reply in replies:
            reply_id=reply.get('id')
            reply_id_list.append(reply_id)

        return reply_id_list
    else:
        return []

In [ ]:
def find_total_num_pages(starting_url):
    soup = simple_crawl(starting_url)
    next_page = soup.find_all("ul", class_="pagination")
    
    for page in next_page:
        pages = page.find_all("a")
        total_num_pages = pages[-2].get("href")
        return int(total_num_pages[-1])

In [ ]:
def calling_functions(commentators, comments_dictionary, comment_number):
    
    for comment in commentators:
        
        comment_id_f = comment_id(comment)
        name_f =  name(comment)
        timedate_f = datetime(comment)
        likes_count_f = likes_count(comment)
        quoted_comment_f = quoted_comment(comment)
        actual_comment_f, actual_comment_str =  actual_comment(comment)
        reply_ids_f = reply_ids(comment)


        comments_dictionary[comment_number] = [comment_id_f,name_f[0], timedate_f, 
                                      likes_count_f, quoted_comment_f[0],
                                       actual_comment_str, actual_comment_f, reply_ids_f]
        comment_number += 1
    
    return comments_dictionary, comment_number

## Function to scrape a particular open data policy

In [ ]:
def scrape_all_pages(starting_url):
    '''
    This functions scrapes the required data from all the webpages for a PARTICULAR starting url of a Madison webpage.
    '''
    comments_dictionary = {}
    num_pages_to_crawl = find_total_num_pages(starting_url)
    
    if num_pages_to_crawl is None:
        num_pages_to_crawl = 1
        
    loop_number = 0
    comment_number = 1

    while loop_number < num_pages_to_crawl:
    
        soup = simple_crawl(starting_url)
        commentators = soup.findAll("li", class_="comment floating-card")
        
        comments_dictionary, comment_number = calling_functions(commentators, comments_dictionary, comment_number)
        
        next_page = soup.find_all("ul", class_="pagination")
        for page in next_page:
            pages = page.find_all("a")
            starting_url = pages[-1].get("href")
        
        loop_number += 1
        
    return comments_dictionary

In [ ]:
comments_dictionary = scrape_all_pages(starting_url)

In [ ]:
comments_dictionary

In [ ]:
def convert_dict_pandas(comments_dictionary):
    
    df = pd.DataFrame(comments_dictionary)
    df = df.transpose()
    df.columns = ['comment_id', 'author', 'datetime', 'num_likes', 'quoted_text', 'comment_text', 'comment_text_aux', 'reply_ids']

    # converting str datetime into datetime pandas format
    df['datetime'] = pd.to_datetime(df['datetime'])
    return df

In [2]:
convert_dict_pandas(comments_dictionary)

NameError: name 'convert_dict_pandas' is not defined

In [3]:
temp = convert_dict_pandas(comments_dictionary)

NameError: name 'convert_dict_pandas' is not defined

In [4]:
# temp['quoted_text'].dropna().map(lambda y: y.replace('<p>','').replace('</p>','')).apply(lambda x: pd.Series(x).value_counts()).sum()
temp['quoted_text'].dropna().apply(lambda x: pd.Series(x.lower().split()).value_counts()).sum().sort_values(ascending=False).head()

NameError: name 'temp' is not defined

# Analysis of comments

## 1) finding out most common words in quoted texts (policy sections) and actual comments

### -most common words in quoted texts

In [5]:
#list_qt = temp['quoted_text'].dropna().tolist()

In [6]:
string_quoted_text = temp['quoted_text'].str.cat(sep=' ').lower()

NameError: name 'temp' is not defined

In [7]:
words = nltk.tokenize.word_tokenize(string_quoted_text)
good_words = [i for i in words if i not in stop] # stop words

NameError: name 'string_quoted_text' is not defined

In [8]:
fdist = FreqDist(good_words)

NameError: name 'good_words' is not defined

In [9]:
common_quoted_text_20 = fdist.most_common(20)

NameError: name 'fdist' is not defined

In [10]:
common_quoted_text_20[:10]

NameError: name 'common_quoted_text_20' is not defined

In [11]:
fdist.plot(20) #resize the plot

NameError: name 'fdist' is not defined

### -most common words in actual comments

In [12]:
string_actual_comments = temp['comment_text'].str.cat(sep=' ').lower()

NameError: name 'temp' is not defined

In [13]:
words_comments = nltk.tokenize.word_tokenize(string_actual_comments)
good_words_comments = [i for i in words_comments if i not in stop] # stop words

NameError: name 'string_actual_comments' is not defined

In [14]:
fdist_comments = FreqDist(good_words_comments)

NameError: name 'good_words_comments' is not defined

In [15]:
common_comment_text_30 = fdist_comments.most_common(30)

NameError: name 'fdist_comments' is not defined

In [16]:
common_comment_text_30[:30]

NameError: name 'common_comment_text_30' is not defined

In [17]:
fdist_comments.plot(30) #resize the plot

NameError: name 'fdist_comments' is not defined

## 2) finding out most common quoted text n-grams

### method 1

In [18]:
string_quoted_text_list = temp['quoted_text'].dropna().tolist()

NameError: name 'temp' is not defined

In [19]:
import re
from nltk.util import ngrams
 
from collections import Counter
 
non_speaker = re.compile('[A-Za-z]+: (.*)')
 
def extract_phrases(text, phrase_counter, length):
    for sent in nltk.sent_tokenize(text):
        strip_speaker = non_speaker.match(sent)
        if strip_speaker is not None:
            sent = strip_speaker.group(1)
        words = nltk.word_tokenize(sent)
        for phrase in ngrams(words, length):
            phrase_counter[phrase] += 1
 
phrase_counter = Counter()
 
#with open("data/import/sentences.csv", "r") as sentencesfile:
#reader = csv.reader(string_quoted_text, delimiter=",")
#reader.next()
for sentence in string_quoted_text_list:
    extract_phrases(sentence, phrase_counter, 4)
 
most_common_phrases = phrase_counter.most_common(15)

for k,v in most_common_phrases:
    print ('{0: <5}'.format(v), k)

# decide on the limit of words frequency (4?)
# now do a jaro wrinkler score to bring the text together

NameError: name 'string_quoted_text_list' is not defined

### finding out negative and positive sentiments for popular policy sections

In [20]:
for phrase in most_common_phrases:
    string_phrase = ' '.join(phrase[0])
    print(string_phrase)

    #print(temp[temp['quoted_text'].astype(str).str.contains(string_phrase)])
    #print("-----")

NameError: name 'most_common_phrases' is not defined

In [21]:
actual_comment_filter = temp[temp['quoted_text'].astype(str).str.contains('Open Data Governance Committee')]
comment_list_sentiment = actual_comment_filter['comment_text'].tolist()

NameError: name 'temp' is not defined

In [ ]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(' ]
neutral_vocab = [ 'movie','the','sound','was','is','actors','did','know','words','not' ]
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
 
train_set = negative_features + positive_features + neutral_features
 
classifier = NaiveBayesClassifier.train(train_set) 
 
# Predict
neg = 0
pos = 0
sentence = "Awesome movie, I liked it"
sentence = sentence.lower()
words = sentence.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1
 
print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))

### method 2

In [371]:
#import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# change this to read in your data
finder = TrigramCollocationFinder.from_words(words)

# only trigrams that appear 3+ times
finder.apply_freq_filter(4) 

# return the 10 n-grams with the highest PMI
print(finder.nbest(bigram_measures.pmi, 20))

#for k,v in finder.ngram_fd.items():
#  print(k,v)

[('data', 'governance', 'committee'), ('open', 'data', 'governance'), ('the', 'open', 'data')]
----------


### method 3

In [368]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.collocations import *
from nltk.probability import FreqDist
import nltk

string_quoted_text_1 = string_quoted_text
vect = CountVectorizer(ngram_range=(1,5))
analyzer = vect.build_analyzer()
listNgramQuery = analyzer(string_quoted_text_1)
listNgramQuery.reverse()
#print ("listNgramQuery=", listNgramQuery)
NgramQueryWeights = nltk.FreqDist(listNgramQuery)
print ("\nNgramQueryWeights=", NgramQueryWeights.most_common(100))

#temp[].dropna().apply(lambda x: pd.Series(x.lower().split()).value_counts()).sum().sort_values(ascending=False)


NgramQueryWeights= [('the', 35), ('data', 30), ('and', 27), ('of', 22), ('open data', 15), ('open', 15), ('for', 14), ('that', 13), ('to', 11), ('on', 11), ('or', 8), ('the open data', 7), ('the open', 7), ('city', 7), ('public', 7), ('open data governance committee', 6), ('data governance committee', 6), ('open data governance', 6), ('governance committee', 6), ('data governance', 6), ('committee', 6), ('governance', 6), ('non', 6), ('of the', 5), ('the public', 5), ('in', 5), ('any', 5), ('policy', 5), ('is', 5), ('records', 5), ('information', 5), ('the open data governance committee', 4), ('the open data governance', 4), ('the data', 4), ('policy concerns', 4), ('develop and', 4), ('implementation', 4), ('concerns', 4), ('access', 4), ('ensure', 4), ('use', 4), ('develop', 4), ('datasets', 4), ('on internal and external needs', 3), ('based on internal and external', 3), ('data based on internal and', 3), ('the data based on internal', 3), ('update the data based on', 3), ('periodi

## Function to scrape all open data policies

In [598]:
def go_all_madison_websites(starting_urls_list):
    '''
    This function scrapes the comments from all the Madison related webistes for cities that have
    launched open data policies and converts them into pandas dataframe objects.
    
    Returns:
        DataFrame object for each city's open data policy.
    '''
    
    all_madison_websites_dfs = []
    for starting_url in starting_urls_list:
        #print(starting_url)
        comments_dictionary = scrape_all_pages(starting_url)
        get_dict_to_pandas = convert_dict_pandas(comments_dictionary)
        relative_url = starting_url.split('/')[-1]
        all_madison_websites_dfs.append((relative_url, get_dict_to_pandas))
        
    return all_madison_websites_dfs

In [605]:
all_madison_dfs = go_all_madison_websites(starting_urls_list)

In [748]:
#for city_name, df in all_madison_dfs:
#    print(df)

In [761]:
def name_tags(starting_url):
    soup = simple_crawl(starting_url)
    commentators = soup.findAll("li", class_="comment floating-card")    
    
    for comment in commentators:
        
        #------------finding names-------------#
        
        name_date = comment.find("div", class_="media-body media-middle")
        #print(type(names))
        #break
        
        #for name in names:
        #    print(name)
        
        name = name_date.find("span")
        #print(name.string.strip())
        
        #for name in names:
        
        #------------finding datetime-------------#
        
        time = name_date.find("time")
        time_update = time.get("datetime")[:-6]
        
        #time_update_1 = time_update.replace((/[A-Z]/), ' ')
        time_update_1 = time_update.replace("T", " ")
        #print(time_update_1)
         
        
        #------------finding comment_id-------------#
        
        #comment_id=comment.find("li").extract()
        comment_id=comment.get('id')
        #print(comment_id)
        
        #------------finding likes(if any)-------------#
        
        likes = name_date.find("span", class_="action-count")
        #print(likes.string.strip())
        
        #------------finding quoted comment-------------#
        
        comments = comment.find("div", class_="comment-content")
        quoted_comment = comments.find("blockquote")
        if quoted_comment is not None:
            quoted_comment_final = quoted_comment.text
            #print(quoted_comment_final)
    
        
        #------------finding actual comment-------------#
        comments = comment.find("div", class_="comment-content")
        #children = comments.children
        children = comments.find_all("p", recursive=False)
        #print(children)
        
        complete_str = ''
        
        for child in children:
            #print(child)
            complete_str += str(child)
        print("STR",complete_str)
            
        #number = 1
        #for actual_comment in children:
            #print("ACT " + str(number), actual_comment)
            
            #if len(actual_comment) > 1:
            #    print(actual_comment)
            #else:
                #print(actual_comment)
            #number += 1
        
        
        #for child in children:
        #    print(type(child))
            
            
            
            #actual_comment = child.string
            #print(actual_comment)
            #if actual_comment is not None:
                
                #actual_comment_final = actual_comment
                #print(actual_comment_final)
                
                
        #------------finding reply(or replies') id(s)-------------#
        
        replies = comment.find_all("div", class_="comment")
        if replies is not None:
            for reply in replies:
                reply_id=reply.get('id')
                #print(reply_id)
        
    #------------finding the next page to crawl-------------#
    
    next_page = soup.find_all("ul", class_="pagination")
    
    
    for i in next_page:
        pages = i.find_all("a")
        #print(pages)
        #print(pages[-1].get("href"))
        
        num_pages = pages[-2].get("href")
        #print(num_pages[-1])

In [762]:
name_tags(starting_url)

STR <p>I am in support of the proposed policy to further enable data and subsequent knowledge sharing. I agree with fellow contributors that it would be beneficial to see more detailed outlines for the specific process that is going to enable community members to contribute to the Open Data portal by creating, uploading and verifying data sets to be published. Myself and colleagues have created a number of valuable data sets that could be beneficial across city departments, community organizations and individual interests. Understanding that only data sets meeting the standards set forth as a result of this policy will be accepted, we can collaboratively drive the understanding of our community from all perspectives.</p>
STR <p>I would be interested to see this more finely documented. It would be great to have a living, breathering process that community members could submit data sets that they have created (required to meet the data standards set forth by the Open Data Policy) for ver